# Using updated astrometry solutions

As part of an effort to improve the absolute astrometry of HST images, STScI has created multiple new astrometric solutions for ACS and WFC3 images.  These solutions are contained in the World Coordinate System (WCS) of the images, as well as headerlet extesions.  This notebook provides an example workflow showing how to use/change to a different solution.

The solutions are contained in the exposure files (`flt.fits` and `flc.fits`), so no extra files are required.  However, these updates were implemented in December 2019 as a part of HST Data Processing version 2019.5.  If data were downloaded before that, they can either be redownloaded (and will contain the new solutions) from mast.stsci.edu or via astroquery, or can be updated by connecting to the database cotaining solutions (shown below).

For more information, see the overview page here: https://outerspace.stsci.edu/pages/viewpage.action?spaceKey=HAdP&title=Improvements+in+HST+Astrometry


<div class="alert alert-block alert-info">
<b>REQUIREMENT:</b> This notebook was designed around using stwcs version 1.5.3, and python version 3.6.  If versions older than this are used <b>the software will likely not work</b>!
</div>

<div class="alert alert-block alert-warning">
<b>NOTE:</b> The new solutions are used by default by the MAST pipeline!  Some datasets may have certain solutions  such as fitting to Gaia DR2, though other datasets (even in the same visit) may not!  Thus, it is crucial to check the which solution (WCS) is active!  The easiest way to check which solution is active is to check the WCSNAME keyword in the header of the SCI extensions.
</div>

# Table Of Contents

 0. [Example Data Download](#0.-Example-Data-Download)
 1.  [New Extensions on FITS Files](#1.-New-extensions-on-fits-files)
 2. [Exploring different solutions](2.-Exploring-different-solutions)
 3. [Applying a headerlet to the science extensions](#3.-Applying-a-headerlet-to-the-science-extensions)
 4. [Restoring the "old" solution](#4.-Restoring-the-"old"-solution)
 5. [Inspecting alignment](#5.-Inspecting-alignment-(optional))
 6. [Changing the WCS of drizzled images](#6.-Changing-the-WCS-of-drizzled-images)

In [1]:
import numpy as np
import os
import shutil
import matplotlib.pyplot as plt

from astropy.io import fits
from astropy.wcs import WCS
from astropy.visualization import PercentileInterval, ImageNormalize, LogStretch
from astroquery.mast import Observations
from stwcs.wcsutil import headerlet
from stwcs import updatewcs
from drizzlepac.align import generate_astrometric_catalog
from drizzlepac import astrodrizzle

%matplotlib notebook

The following task in the stsci.skypac package can be run with TEAL:
                                    skymatch                                    


The following tasks in the drizzlepac package can be run with TEAL:
    astrodrizzle       config_testbed      imagefindpars           mapreg       
       photeq            pixreplace           pixtopix            pixtosky      
  refimagefindpars       resetbits          runastrodriz          skytopix      
     tweakback            tweakreg           updatenpol


Because some steps in this notebook require access to reference files, we will create a temporary 'iref' directory here to place these reference files after download. This step is typically done by defining the 'iref' path in your bash profile so that all reference files for all datasets can be in one static location, but for the portability of this notebook we will just create a temporary directory. Please see the ['Initalization' notebook](../Initialization/Initialization.ipynb) for more information.

In [2]:
os.environ['CRDS_SERVER_URL'] = 'https://hst-crds.stsci.edu'
os.environ['CRDS_PATH'] = os.path.abspath(os.path.join('.', 'reference_files'))

os.environ['iref'] = os.path.abspath(os.path.join('.', 'reference_files', 'references', 'hst', 'wfc3')) + os.path.sep

## 0. Example Data Download

Lets find some example HST data from MAST and download it:

In [3]:
obsTable = Observations.query_criteria(project='HST', proposal_id='14689', obs_id='ID7307030')
products = Observations.get_product_list(obsTable)
filtered_products = Observations.filter_products(products, mrp_only=False, productSubGroupDescription=['FLC', 'ASN'])
filtered_products

obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid,dataRights,calib_level
str8,str3,str5,str37,str64,str1,str69,str9,str28,str11,str1,str7,str19,str5,str52,int64,str8,str6,int64
25874804,HST,image,id7307xfq,DADS FLC file - CTE-corrected calibrated exposure ACS/WFC3,S,mast:HST/product/id7307xfq_flc.fits,SCIENCE,--,FLC,--,CALWF3,3.7.1 (Oct-18-2023),14689,id7307xfq_flc.fits,169032960,25874840,PUBLIC,2
25874805,HST,image,id7307xhq,DADS FLC file - CTE-corrected calibrated exposure ACS/WFC3,S,mast:HST/product/id7307xhq_flc.fits,SCIENCE,--,FLC,--,CALWF3,3.7.1 (Oct-18-2023),14689,id7307xhq_flc.fits,169032960,25874840,PUBLIC,2
25874806,HST,image,id7307xlq,DADS FLC file - CTE-corrected calibrated exposure ACS/WFC3,S,mast:HST/product/id7307xlq_flc.fits,SCIENCE,--,FLC,--,CALWF3,3.7.1 (Oct-18-2023),14689,id7307xlq_flc.fits,169032960,25874840,PUBLIC,2
25874807,HST,image,id7307xpq,DADS FLC file - CTE-corrected calibrated exposure ACS/WFC3,S,mast:HST/product/id7307xpq_flc.fits,SCIENCE,--,FLC,--,CALWF3,3.7.1 (Oct-18-2023),14689,id7307xpq_flc.fits,169032960,25874840,PUBLIC,2
25874840,HST,image,id7307030,DADS ASN file - Association ACS/WFC3/STIS,D,mast:HST/product/id7307030_asn.fits,AUXILIARY,Minimum Recommended Products,ASN,--,CALWF3,3.7.1 (Oct-18-2023),14689,id7307030_asn.fits,11520,25874840,PUBLIC,3
26303775,HST,image,hst_14689_07_wfc3_uvis_f814w_id7307xh,HAP flat-field product,D,mast:HST/product/hst_14689_07_wfc3_uvis_f814w_id7307xh_flc.fits,SCIENCE,--,FLC,--,HAP-SVM,DrizzlePac 3.6.2,14689,hst_14689_07_wfc3_uvis_f814w_id7307xh_flc.fits,169148160,25874840,PUBLIC,2
26303830,HST,image,hst_14689_07_wfc3_uvis_f814w_id7307xl,HAP flat-field product,D,mast:HST/product/hst_14689_07_wfc3_uvis_f814w_id7307xl_flc.fits,SCIENCE,--,FLC,--,HAP-SVM,DrizzlePac 3.6.2,14689,hst_14689_07_wfc3_uvis_f814w_id7307xl_flc.fits,169148160,25874840,PUBLIC,2
26303880,HST,image,hst_14689_07_wfc3_uvis_f814w_id7307xp,HAP flat-field product,D,mast:HST/product/hst_14689_07_wfc3_uvis_f814w_id7307xp_flc.fits,SCIENCE,--,FLC,--,HAP-SVM,DrizzlePac 3.6.2,14689,hst_14689_07_wfc3_uvis_f814w_id7307xp_flc.fits,169148160,25874840,PUBLIC,2
26303905,HST,image,hst_14689_07_wfc3_uvis_f814w_id7307xf,HAP flat-field product,D,mast:HST/product/hst_14689_07_wfc3_uvis_f814w_id7307xf_flc.fits,SCIENCE,--,FLC,--,HAP-SVM,DrizzlePac 3.6.2,14689,hst_14689_07_wfc3_uvis_f814w_id7307xf_flc.fits,169148160,25874840,PUBLIC,2


In [4]:
dl_tbl = Observations.download_products(filtered_products, mrp_only=False)
dl_tbl

 [Done]


 [Done]


 [Done]


 [Done]


 [Done]


 [Done]


 [Done]


 [Done]


 [Done]


Local Path,Status,Message,URL
str103,str8,object,object
./mastDownload/HST/id7307xfq/id7307xfq_flc.fits,COMPLETE,None,None
./mastDownload/HST/id7307xhq/id7307xhq_flc.fits,COMPLETE,None,None
./mastDownload/HST/id7307xlq/id7307xlq_flc.fits,COMPLETE,None,None
./mastDownload/HST/id7307xpq/id7307xpq_flc.fits,COMPLETE,None,None
./mastDownload/HST/id7307030/id7307030_asn.fits,COMPLETE,None,None
./mastDownload/HST/hst_14689_07_wfc3_uvis_f814w_id7307xh/hst_14689_07_wfc3_uvis_f814w_id7307xh_flc.fits,COMPLETE,None,None
./mastDownload/HST/hst_14689_07_wfc3_uvis_f814w_id7307xl/hst_14689_07_wfc3_uvis_f814w_id7307xl_flc.fits,COMPLETE,None,None
./mastDownload/HST/hst_14689_07_wfc3_uvis_f814w_id7307xp/hst_14689_07_wfc3_uvis_f814w_id7307xp_flc.fits,COMPLETE,None,None
./mastDownload/HST/hst_14689_07_wfc3_uvis_f814w_id7307xf/hst_14689_07_wfc3_uvis_f814w_id7307xf_flc.fits,COMPLETE,None,None


Now to make the paths easier to work with, lets move those files from their default download locations into the same location as this notebook:

In [5]:
for f in dl_tbl['Local Path']:
    filename = os.path.split(f)[-1]
    if not os.path.exists(filename):
        shutil.move(f, '.')

And then download all of the necessary reference files for the images using CRDS (this uses the CRDS paths defined at the top of the notebook, although you would probably want to use a different path for your own data as described above):

In [6]:
for row in filtered_products:
    if row['productSubGroupDescription'] == 'FLC':
        os.system('crds bestrefs --files {}_flc.fits --sync-references=1 --update-bestrefs'.format(row['obs_id']))

CRDS - INFO -  Fetching  /home/runner/work/hst_notebooks/hst_notebooks/notebooks/DrizzlePac/using_updated_astrometry_solutions/reference_files/mappings/hst/hst_wfpc2_wf4tfile_0250.rmap      678 bytes  (1 / 141 files) (0 / 1.7 M bytes)


CRDS - INFO -  Fetching  /home/runner/work/hst_notebooks/hst_notebooks/notebooks/DrizzlePac/using_updated_astrometry_solutions/reference_files/mappings/hst/hst_wfpc2_shadfile_0250.rmap      977 bytes  (2 / 141 files) (678 / 1.7 M bytes)


CRDS - INFO -  Fetching  /home/runner/work/hst_notebooks/hst_notebooks/notebooks/DrizzlePac/using_updated_astrometry_solutions/reference_files/mappings/hst/hst_wfpc2_offtab_0250.rmap      642 bytes  (3 / 141 files) (1.7 K / 1.7 M bytes)


CRDS - INFO -  Fetching  /home/runner/work/hst_notebooks/hst_notebooks/notebooks/DrizzlePac/using_updated_astrometry_solutions/reference_files/mappings/hst/hst_wfpc2_maskfile_0250.rmap      685 bytes  (4 / 141 files) (2.3 K / 1.7 M bytes)


CRDS - INFO -  Fetching  /home/runner/work/hst_notebooks/hst_notebooks/notebooks/DrizzlePac/using_updated_astrometry_solutions/reference_files/mappings/hst/hst_wfpc2_idctab_0250.rmap      696 bytes  (5 / 141 files) (3.0 K / 1.7 M bytes)


...

CRDS - INFO -  0 errors
CRDS - INFO -  0 warnings
CRDS - INFO -  2 infos


CRDS - INFO -  No comparison context or source comparison requested.
CRDS - INFO -  ===> Processing hst_14689_07_wfc3_uvis_f814w_id7307xp_flc.fits


CRDS - INFO -  0 errors
CRDS - INFO -  0 warnings
CRDS - INFO -  2 infos


CRDS - INFO -  No comparison context or source comparison requested.
CRDS - INFO -  ===> Processing hst_14689_07_wfc3_uvis_f814w_id7307xf_flc.fits


CRDS - INFO -  0 errors
CRDS - INFO -  0 warnings
CRDS - INFO -  2 infos


## 1. New extensions on fits files

Using `fits.info` prints basic information about the extensions in a fits file.  In the following examples, we will show operations for one file, though performing the same operations for multiple files simply requires looping.

In [7]:
filename = 'id7307xfq_flc.fits'

If no extensions named `HDRLET` appear, then solutions can be downloaded from the database (if available) and automatically appended to the fits files using the following command:

In [8]:
updatewcs.updatewcs(filename, use_db=True)

AstrometryDB service available...


Deleted all instances of WCS with key A in extensions [1, 4]
Deleted all instances of WCS with key B in extensions [1, 4]
Wcskey 'O' is reserved for the original WCS and should not be deleted.
INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here

INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                 [astropy.wcs.wcs]
- IDCTAB: Distortion

Updating astrometry for id7307xfq


Accessing AstrometryDB service :


...

Retrieving astrometrically-updated WCS "IDC_2731450pi-FIT_SVM_GAIADR2" for observation "id7307xfq"


Updating id7307xfq with:


Replacing primary WCS with


	Headerlet with WCSNAME=IDC_2731450pi-FIT_SVM_GAIADR2


['id7307xfq_flc.fits']

The updated solutions should then show up as extra `HDRLET` extensions

In [9]:
fits.info(filename)

Filename: id7307xfq_flc.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     304   ()      
  1  SCI           1 ImageHDU       207   (4096, 2051)   float32   
  2  ERR           1 ImageHDU        50   (4096, 2051)   float32   
  3  DQ            1 ImageHDU        42   (4096, 2051)   int16   
  4  SCI           2 ImageHDU       205   (4096, 2051)   float32   
  5  ERR           2 ImageHDU        50   (4096, 2051)   float32   
  6  DQ            2 ImageHDU        42   (4096, 2051)   int16   
  7  HDRLET        1 HeaderletHDU     18   ()      
  8  HDRLET        2 HeaderletHDU     26   ()      
  9  HDRLET        3 HeaderletHDU     26   ()      
 10  HDRLET        4 HeaderletHDU     26   ()      
 11  HDRLET        5 HeaderletHDU     26   ()      
 12  HDRLET        6 HeaderletHDU     26   ()      
 13  WCSCORR       1 BinTableHDU     59   14R x 24C   [40A, I, A, 24A, 24A, 24A, 24A, D, D, D, D, D, D, D, D, 24A, 24A, D, D, D, D, J, 40A, 12

As seen above, there are new `HDRLET` extensions in the fits files (compared to the pre-2019.3 products).  These extensions each contain information used to construct a World Coordinate System (WCS), which is used to transform image coordinates into physical (sky) coordinates.  Each of these WCS's represent an astrometric solution derived in a different way.

## 2. Exploring different solutions

Each HDRLET extension contains information describing the solution used in its creation. To investigate this we first programmatically obtain the extension numbers of the HDRLETs.

In [10]:
ext_indices = headerlet.find_headerlet_HDUs(filename, strict=False)

# To show it's consistent with the fits.info from above
print(ext_indices)

[7, 8, 9, 10, 11, 12, 14, 15]


We can then loop through these extensions to see what the WCS solutions are.

In [11]:
hdu = fits.open(filename)
print('Ext\tWCSNAME')
for ext_ind in ext_indices:
    print(ext_ind, '\t', hdu[ext_ind].header['WCSNAME'])

hdu.close()

Ext	WCSNAME
7 	 OPUS
8 	 IDC_2731450pi
9 	 IDC_2731450pi-GSC240
10 	 IDC_2731450pi-FIT_REL_GAIADR2
11 	 id7307xfq_flt_IDC_2731450pi-GSC240-hlet.fits
12 	 IDC_2731450pi-FIT_SVM_GAIADR2
14 	 IDC_2731450pi
15 	 IDC_2731450pi-FIT_REL_GAIAeDR3


Alternatively, we can use `get_headerlet_kw_names`:

In [12]:
new_wcsnames = headerlet.get_headerlet_kw_names(filename, kw='WCSNAME')
new_wcsnames

['OPUS',
 'IDC_2731450pi',
 'IDC_2731450pi-GSC240',
 'IDC_2731450pi-FIT_REL_GAIADR2',
 'id7307xfq_flt_IDC_2731450pi-GSC240-hlet.fits',
 'IDC_2731450pi-FIT_SVM_GAIADR2',
 'IDC_2731450pi',
 'IDC_2731450pi-FIT_REL_GAIAeDR3']

We can write this into a convenience function:

In [13]:
def get_hdrlet_wcsnames(filename):
    """Print and return list of WCS names in HDRLET extensions of fits file"""
    hdu = fits.open(filename)
    ext_indices = headerlet.find_headerlet_HDUs(filename, strict=False)

    print('Ext\tWCSNAME')
    new_wcsnames = []
    for ext_ind in ext_indices:
        name = hdu[ext_ind].header['WCSNAME']
        print(ext_ind, '\t', name)
        new_wcsnames.append(name)
        
    hdu.close()
    return new_wcsnames

In [14]:
new_wcsnames = get_hdrlet_wcsnames(filename)

Ext	WCSNAME
7 	 OPUS
8 	 IDC_2731450pi
9 	 IDC_2731450pi-GSC240
10 	 IDC_2731450pi-FIT_REL_GAIADR2
11 	 id7307xfq_flt_IDC_2731450pi-GSC240-hlet.fits
12 	 IDC_2731450pi-FIT_SVM_GAIADR2
14 	 IDC_2731450pi
15 	 IDC_2731450pi-FIT_REL_GAIAeDR3


We can also see which solution is the "active" solution (the one currently applied to the SCI extensions):

In [15]:
current_wcs = fits.getval(filename, 'WCSNAME', ext=('SCI', 1))

print(current_wcs)

IDC_2731450pi-FIT_SVM_GAIADR2


The nature of each solution is described here: https://drizzlepac.readthedocs.io/en/latest/astrometry.html#interpreting-wcs-names

## 3. Applying a headerlet to the science extensions

To apply/activate one of the other solutions, the `restore_from_headerlet()` function can be used.  This function applies the WCS contained in a HDRLET extension to all SCI extensions of the image.  Doing this requires knowing which solution should be applied, which can be obtained in multiple ways.

<div class="alert alert-block alert-warning">
NOTE: This is especially useful in cases where some of the exposures in a visit will have solutions that are aligned to Gaia, but others won't.  This is true for grism images in the same visit as direct images, or shallow/deep exposure combinations.
<div>

For instance, if the desired solution is `IDC_2731450pi-GSC240`, we can find the `EXTVER` of the corresponding HDRLET from the list of wcs names we generated before.

In [16]:
# Gets the index of list element with value 'IDC_2731450pi-GSC240'
# The index in this list + 1 is the same as the EXTVER of the corresponding HDRLET
# We need to add 1 because lists are 0-indexed, while EXTVER's are 1 indexed
chosen_ext = new_wcsnames.index('IDC_2731450pi-GSC240') + 1

In [17]:
headerlet.restore_from_headerlet(filename, hdrext=('HDRLET', chosen_ext), archive=False, force=False)

In this case we set `archive` keyword argument to `False`.  Setting `archive` to True will preserve the currently active WCS as a new HDRLET extension on the file.  Since in our case the current solution already has a HDRLET, we do not need to archive it.  This may be useful in some cases, such as when the image has been manually aligned/transformed, and keeping a record of that solution is desired.



We can check that the solution was applied:

In [18]:
current_wcs = fits.getval(filename, 'WCSNAME', ext=('SCI', 1))

print(current_wcs)

IDC_2731450pi-GSC240


We can also apply the solution via the HDRNAME:

In [19]:
hdrlet_hdrnames = headerlet.get_headerlet_kw_names(fits.open(filename), 'HDRNAME')
desired_hdrname = hdrlet_hdrnames[new_wcsnames.index('IDC_2731450pi-GSC240')]

In [20]:
print(desired_hdrname)

id7307xfq_flt_IDC_2731450pi-GSC240


In [21]:
headerlet.restore_from_headerlet(filename, hdrname=desired_hdrname, archive=False, force=False)

We can also apply some logic to get the `hdrext` programatically.  For instance, if we only wanted the `IDC` (distortion calibrated) solution with the `GSC240` tag (indicating that the guide star positions had been updated), we can do the following:

In [22]:
for i, wcsname in enumerate(new_wcsnames):
    if 'IDC' in wcsname and 'GSC240' in wcsname:
        chosen_ext = i + 1 # Add one due to 0 indexing of enumerate vs 1 indexing of EXTVER
        break

print('The desired extension is:', ('HDRLET', chosen_ext))

The desired extension is: ('HDRLET', 3)


Finding the solution this way can be easier, as it doesn't require a full typing out of the IDCTAB name.  However, in the future, if new IDCTABs are created, there may be multiple solutions matching the criteria above, and more sophisticated logic will need to be applied.

## 4. Restoring the "old" solution

If the original solution is desired, it can be restored using the methods shown above, by replacing the WCSNAME simply with `IDC_2731450pi`, or whatever the name of the IDCTAB for that file is.  To get that name, the following procedure can be done:


<div class="alert alert-block alert-warning">
<b>NOTE:</b> Data taken after October 2017 may not have a solution of the form IDC_xxxxxxxxx, as the pointing information of the telescope was already calculated using GSC 2.4.0.  As such, the "old" solution may be of the same form as the a priori solution, i.e.: IDC_xxxxxxxxx-GSC240.
</div>

In [23]:
idc_file = fits.getval(filename, 'IDCTAB')
idc_filename = idc_file.split('$')[-1]
idc_wcsname = 'IDC_' + idc_filename.replace('_idc.fits', '')

In [24]:
print(idc_wcsname)

IDC_2731450pi


Restoring the solution is then done the same as above:

In [25]:
chosen_ext = new_wcsnames.index(idc_wcsname) + 1

In [26]:
headerlet.restore_from_headerlet(filename, hdrext=('HDRLET', chosen_ext), archive=False, force=False)

In [27]:
current_wcs = fits.getval(filename, 'WCSNAME', ext=('SCI', 1))

print(current_wcs)

IDC_2731450pi


Alternatively, `updatewcs` can be run on the image, with the `use_db` flag set to `False`:

In [28]:
updatewcs.updatewcs(filename, use_db=False)

Deleted all instances of WCS with key A in extensions [1, 4]
Wcskey 'O' is reserved for the original WCS and should not be deleted.
INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consiste

INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                 [astropy.wcs.wcs]
- IDCTAB: Distortion

['id7307xfq_flc.fits']

In [29]:
current_wcs = fits.getval(filename, 'WCSNAME', ext=('SCI', 1))

print(current_wcs)

IDC_2731450pi


## 5. Inspecting alignment (optional)

While the solutions should improve the astrometry of the images, they may not have perfect absolute astrometry.  To check the quality of the absolute astrometry.  To do this, we can overplot the positions of the Gaia sources on the images.  If the absolute astrometry is correct, the Gaia positions will project on top of the sources in the images.  The WCS solutions that contain 'FIT-GAIA' in the WCSNAME should have the closest alignment, while the GSC-240 and baseline IDC solution will likely have small offsets.

We first need to get a listing of the Gaia sources:

In [30]:
ast_tbl = generate_astrometric_catalog('id7307xfq_flc.fits', output=True) 
ast_tbl

RA,DEC,mag,objID
float64,float64,float64,int64
80.5730899119131,-67.9474991750782,--,4658741306468400256
80.5439170969378,-67.9438011573397,--,4658741370926206336
80.5924054598397,-67.9544180247475,--,4658741306468435328
80.5214509397304,-67.9429440875399,--,4658741581346201600
80.5087676329488,-67.9648937699992,21.59373,4658741169049735680
80.5396243528847,-67.9352493940572,21.54121,4658741478269158016
80.5510482118061,-67.9703065150295,21.41451,4658740962871024768
80.6007478478036,-67.9510988199131,21.33524,4658741405311718400
80.5812549526072,-67.9517424214045,21.24463,4658741302232469248


We then need the WCS of the images, to project the Gaia positions onto the image coordinate frame.

In [31]:
hdu = fits.open(filename)
w = WCS(hdu['SCI', 1].header, hdu)
data = hdu['SCI', 1].data

In [32]:
x, y = w.all_world2pix(np.array([ast_tbl['RA'], ast_tbl['DEC']]).T, 0).T

In [33]:
fig = plt.figure(figsize=(10, 5))
ax = plt.subplot(projection=w) # matplotlib has support for axes using WCS projections!

ax.imshow(data, origin='lower',
          norm=ImageNormalize(data, interval=PercentileInterval(99.65), stretch=LogStretch()))
ax.coords.grid(True, color='white', ls=':', alpha=.6)
ax.autoscale(False)
ax.scatter(x, y, edgecolors='r', facecolor=None, alpha=.8)

<IPython.core.display.Javascript object>

The original (only IDC tab) WCS will likely have an offset between the Gaia positions and the sources.  The GSC-240 solution offset is generally smaller, while any of the `GAIA-FIT` solutions should be well under a pixel.  Using the code in section 3, the offsets can be seen for each available WCS.

## 6. Changing the WCS of drizzled images

Because the drizzling process is directly affected by the WCS's of the input images, the WCS of the drizzled image cannot be changed as simply as shown above for FLC images.  To use an astrometric solution (other than the one  applied to the FLT/FLC at the time of drizzling), the images will have to be re-drizzled after activating the desired WCS in the FLT/FLC images.

<div class="alert alert-block alert-warning">
<b>NOTE:</b> The images input to astrodrizzle should use the same WCS solution, or the drizzling process will produce poor results.
</div>

In order to get the same name for the drizzled file, we can simply run astrodrizzle on the association (asn) file we downloaded before.

In [34]:
astrodrizzle.AstroDrizzle('id7307030_asn.fits', mdriztab=True, build=True, clean=True, preserve=False)

Setting up logfile :  astrodrizzle.log


AstroDrizzle log file: astrodrizzle.log


AstroDrizzle Version 3.5.1 started at: 13:19:53.527 (01/04/2024)


==== Processing Step  Initialization  started at  13:19:53.529 (01/04/2024)


Reading in MDRIZTAB parameters for 4 files


...

                Driz_CR          3.9009 sec.


          Final Drizzle          14.9917 sec.


   ====================          ====================


                  Total          38.3157 sec.


Trailer file written to:  astrodrizzle.log


# About this Notebook

    Updated: June 12, 2023 by A. O'Connor -- STScI WFC3 